In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_csv('/kaggle/train.csv')
train.head()
test = pd.read_csv('/kaggle/public_test.csv')
test.head()

In [ ]:
# tokenizer 사용 
from keras.preprocessing.text import Tokenizer
tk = Tokenizer()

# train과 test 모든 'text' 데이터 숫자화 
tk.fit_on_texts(list(train['text']) + list(test['text']))


tk.word_index 
len(tk.word_index) 

# 각 데이터 프레임 내 텍스트를 숫자화
train_text = tk.texts_to_sequences(train['text'])
test_text = tk.texts_to_sequences(test['text'])


pd.Series(train_text).apply(lambda x : len(x)).max()
pd.Series(train_text).apply(len)


# 숫자로 변한 텍스트의 분포 확인
import matplotlib.pyplot as plt
import seaborn as sns

a, b = plt.subplots(1,1,figsize=(20,12)) # 밑그림
sns.distplot(pd.Series(train_text).apply(len)) # 대다수가 50개 정도 / train을 보면 test도 봐야함



In [ ]:

from keras.preprocessing.sequence import pad_sequences

# 근데 위 빈도 그래프에 따라서 100개에서 짤라도 크게 차이는 없을 것 같음 
pad_train = pad_sequences(train_text, maxlen=100, truncating ='post', padding= 'post') # truncating 기본 값은 pre
pad_test = pad_sequences(test_text, maxlen=100, truncating ='post', padding= 'post') # truncating 기본 값은 pre

In [ ]:
# 스미싱이 전체 데이터 셋과 비교해 6%를 차지

from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(pad_train, train['smishing'], test_size = 0.15, random_state=0, stratify=train['smishing']) #stratify 0,1 비율을 자동으로 맞춰줌

In [ ]:
from keras import Sequential
from keras.layers import Dense, Embedding, Flatten 

# 2) 모델쌓기, 선언
model = Sequential()
model.add(Embedding(448903,25,input_length=100)) #  열심히 찾아나가야 되는 부분

#embedding(단어의 의미를 학습한다), 각 데이터에 접근해 스미싱인가 아닌가 *단어*를 보고 판단해야함 

model.add(Flatten()) # 2차원 -> 1차원 
model.add(Dense(2, activation='softmax')) # 옵션2개 중 1개는 정답 클래스 갯수

model.compile(metrics = ['acc'], optimizer = 'adam', loss='sparse_categorical_crossentropy') # 3가지 옵션 넣어줄거임 * compile, 모델선언 완료

from keras.callbacks import EarlyStopping, ModelCheckpoint
es = EarlyStopping(patience=2) 
mc = ModelCheckpoint('best.h5', save_best_only=True) 

# 3) 모델 학습
# model.fit(x_train, y_train, epochs=100, validation_split=0.1,batch_size=128, callbacks=[es,mc]) #callback #batch size는 32가 기본 / 
# x, y 설정 / 학습 횟수 결정, epoch / 학습(90%), 평가(10%)


model.fit(x_train, y_train, epochs=100, validation_data=(x_valid, y_valid),batch_size=128, callbacks=[es,mc]) #callback #batch size는 32가 기본 /  
# x, y 설정 / 학습 횟수 결정, epoch / 학습(90%), 평가(10%)




In [ ]:
# 최적 가중치 저장된 걸 가져와야함 
model.load_weights('best.h5')

In [ ]:
# predict
result = model.predict(pad_test, batch_size=128) #pad_test는 뒤쪽 짜른거 
result
# 왼쪽 / 오른쪽(), 스미싱 확률

In [ ]:
test['smishing'] = result[:,1]
sub = test[['id','smishing']]
sub.to_csv('baseline_평가셋 수정.csv', index=False)

In [ ]:
first = pd.read_csv('/kaggle/output/working/baseline_평가셋 수정.csv')
second = pd.read_csv('/kaggle/input/taling/baseline_ .csv')
display(first.head())
display(second.head()) # 비교 시, 스미싱을 판단하는 수치 다름
# roc-auc평가방식 -> 확률이 중요한게 아닌, 상대적 순위가 중요해짐. 이에 두개의 앙상블 시, '순위'를 고려해야함 